In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [7]:
# Загрузка файла
file_path = './Titanic.csv'
data = pd.read_csv(file_path)

# Просмотр первых строк и общей информации о данных
data_info = data.info()
data_head = data.head()

data_info, data_head


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


(None,
    PassengerId  Survived  Pclass  \
 0          892         0       3   
 1          893         1       3   
 2          894         0       2   
 3          895         0       3   
 4          896         1       3   
 
                                            Name     Sex   Age  SibSp  Parch  \
 0                              Kelly, Mr. James    male  34.5      0      0   
 1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
 2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
 3                              Wirz, Mr. Albert    male  27.0      0      0   
 4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   
 
     Ticket     Fare Cabin Embarked  
 0   330911   7.8292   NaN        Q  
 1   363272   7.0000   NaN        S  
 2   240276   9.6875   NaN        Q  
 3   315154   8.6625   NaN        S  
 4  3101298  12.2875   NaN        S  )

### Разделение данных на обучающую и тестовую выборки

In [8]:
# Упрощенная предобработка: использование только числовых и категориальных данных без пропусков
data_simple = data[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Survived']].copy()

# Кодирование категориального признака 'Sex'
data_simple['Sex'] = data_simple['Sex'].map({'male': 0, 'female': 1})

# Удаление строк с пропусками
data_simple.dropna(inplace=True)

# Разделение на признаки (X) и целевую переменную (y)
X = data_simple.drop(columns=['Survived'])
y = data_simple['Survived']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((291, 5), (126, 5), (291,), (126,))

### Обучение модели на первоначальных данных без обработки

In [19]:
# Инициализация и обучение логистической регрессии на первоначальных данных
model_baseline = LogisticRegression(random_state=42)
model_baseline.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred_baseline = model_baseline.predict(X_test)

# Оценка метрик
accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
report_baseline = classification_report(y_test, y_pred_baseline, target_names=["Not Survived", "Survived"])

accuracy_baseline

1.0

In [20]:
report_baseline

'              precision    recall  f1-score   support\n\nNot Survived       1.00      1.00      1.00        79\n    Survived       1.00      1.00      1.00        47\n\n    accuracy                           1.00       126\n   macro avg       1.00      1.00      1.00       126\nweighted avg       1.00      1.00      1.00       126\n'

### Обучение модели на обработанных данных и сравнение метрик

In [17]:
# Дополнительная обработка данных: нормализация числовых признаков
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение модели на обработанных данных
model_processed = LogisticRegression(random_state=42)
model_processed.fit(X_train_scaled, y_train)

# Предсказания на тестовой выборке
y_pred_processed = model_processed.predict(X_test_scaled)

# Оценка метрик
accuracy_processed = accuracy_score(y_test, y_pred_processed)
report_processed = classification_report(y_test, y_pred_processed, target_names=["Not Survived", "Survived"])

accuracy_processed

1.0

In [18]:
report_processed

'              precision    recall  f1-score   support\n\nNot Survived       1.00      1.00      1.00        79\n    Survived       1.00      1.00      1.00        47\n\n    accuracy                           1.00       126\n   macro avg       1.00      1.00      1.00       126\nweighted avg       1.00      1.00      1.00       126\n'

In [11]:
# Статистический подход: поиск выбросов с использованием межквартильного размаха (IQR)
Q1 = X_train.quantile(0.25)
Q3 = X_train.quantile(0.75)
IQR = Q3 - Q1

# Определение границ для выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Выбросы по каждому признаку
outliers = ((X_train < lower_bound) | (X_train > upper_bound)).any(axis=1)

# Удаление выбросов
X_train_no_outliers = X_train[~outliers]
y_train_no_outliers = y_train[~outliers]

# Проверка размера данных после удаления выбросов
X_train_no_outliers.shape, y_train_no_outliers.shape

((245, 5), (245,))

In [15]:
# Обучение модели на данных без выбросов
model_no_outliers = LogisticRegression(random_state=42)
model_no_outliers.fit(X_train_no_outliers, y_train_no_outliers)

# Предсказания на тестовой выборке
y_pred_no_outliers = model_no_outliers.predict(X_test)

# Оценка метрик
accuracy_no_outliers = accuracy_score(y_test, y_pred_no_outliers)
report_no_outliers = classification_report(y_test, y_pred_no_outliers, target_names=["Not Survived", "Survived"])

accuracy_no_outliers

1.0

In [16]:
report_no_outliers

'              precision    recall  f1-score   support\n\nNot Survived       1.00      1.00      1.00        79\n    Survived       1.00      1.00      1.00        47\n\n    accuracy                           1.00       126\n   macro avg       1.00      1.00      1.00       126\nweighted avg       1.00      1.00      1.00       126\n'

После обработки данных модель также достигла идеальных метрик:
- **Точность (accuracy)**: 1.0
- Метрики precision, recall, f1-score также равны 1.0.

Это может свидетельствовать о том, что данные слишком простые для текущей задачи.

### Выявление выбросов с использованием статистического подхода и DBSCAN.

In [13]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Стандартизация данных для DBSCAN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Применение DBSCAN
dbscan = DBSCAN(eps=1.5, min_samples=5)
clusters = dbscan.fit_predict(X_train_scaled)

# Определение выбросов (метка -1)
no_outliers_mask = clusters != -1

# Удаление выбросов
X_train_dbscan = X_train[no_outliers_mask]
y_train_dbscan = y_train[no_outliers_mask]

# Проверка размера данных после удаления выбросов
X_train_dbscan.shape, y_train_dbscan.shape

((276, 5), (276,))

### Обучение модели на данных без выбросов, выявленных DBSCAN

In [21]:
# Обучение модели на данных без выбросов (DBSCAN)
model_dbscan = LogisticRegression(random_state=42)
model_dbscan.fit(X_train_dbscan, y_train_dbscan)

# Предсказания на тестовой выборке
y_pred_dbscan = model_dbscan.predict(X_test)

# Оценка метрик
accuracy_dbscan = accuracy_score(y_test, y_pred_dbscan)
report_dbscan = classification_report(y_test, y_pred_dbscan, target_names=["Not Survived", "Survived"])

accuracy_dbscan

1.0

In [22]:
report_dbscan

'              precision    recall  f1-score   support\n\nNot Survived       1.00      1.00      1.00        79\n    Survived       1.00      1.00      1.00        47\n\n    accuracy                           1.00       126\n   macro avg       1.00      1.00      1.00       126\nweighted avg       1.00      1.00      1.00       126\n'

### Выводы:
1. Вне зависимости от обработки данных и удаления выбросов (статистическим подходом или DBSCAN), данные слишком просты для задачи, что приводит к идеальным метрикам.